In [151]:
#Import required libraries for data loading, cleaning and pre processing.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [152]:
#Load the dataset
df = pd.read_csv('train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50



**The dataset consists of 5847 rows and 14 columns**

In [153]:
#Descriptive Statistics
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB
None
        Unnamed: 0         Year  Kilometers_Driven      

## **a) Handling Missing Values**

In [154]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


1. Mileage, Engine, Power, Seats: These columns have only a small portion of missing values relative to the dataset size. Mean imputation has been done here since they are numerical features

2. New_Price: Filling missing values with 0

In [155]:
# Extract numeric values and convert the columns
df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.\d+|\d+)').astype(float)  # Remove non-numeric parts and convert to float
df['Engine'] = df['Engine'].str.extract(r'(\d+)').astype(float)
df['Power'] = df['Power'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [156]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Impute missing values using the mean for Mileage, Engine, Power, Seats
df['Mileage'].fillna(df['Mileage'].mean(), inplace=True)
df['Engine'].fillna(df['Engine'].mean(), inplace=True)
df['Power'].fillna(df['Power'].mean(), inplace=True)
df['Seats'].fillna(df['Seats'].mean(), inplace=True)

In [157]:
# Fill missing values in 'New_Price' with 0 (as a placeholder)
df['New_Price'].fillna(0, inplace=True)

# Function to clean the 'New_Price' column
def clean_new_price(price):
    if isinstance(price, str):  # Check if the value is a string
        if 'Cr' in price:
            return float(price.replace(' Cr', '')) * 100
        elif 'Lakh' in price:
            return float(price.replace(' Lakh', ''))
    return price  # Return as is if it's already numeric or NaN

# Apply the function and convert to float
df['New_Price'] = df['New_Price'].apply(clean_new_price).astype(float)

In [158]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


**We can see that there are no missing values after being imputed.**

## **Step b: The units have already been removed while extracting the numeric columns**



## **C) Label encoding categorical features**

1. Fuel_Type and Transmission were converted into numerical Label encoded columns.

In [159]:
# c) Label encoding for Fuel_Type and Transmission
df['Fuel_Type'] = df['Fuel_Type'].astype('category').cat.codes
df['Transmission'] = df['Transmission'].astype('category').cat.codes

## **d) Creating a New Feature:**
A new column Car_Age was added, representing the car's age (2024 - Year).

In [160]:
# d) Create new feature 'Mileage_per_CC'
df['Mileage_per_CC'] = df['Mileage'] / df['Engine']
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Mileage_per_CC
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,1,First,19.67,1582.0,126.20,5.0,0.00,12.50,0.012434
1,2,Honda Jazz V,Chennai,2011,46000,2,1,First,13.00,1199.0,88.70,5.0,8.61,4.50,0.010842
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,1,First,20.77,1248.0,88.76,7.0,0.00,6.00,0.016643
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,0,Second,15.20,1968.0,140.80,5.0,0.00,17.74,0.007724
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,1,First,23.08,1461.0,63.10,5.0,0.00,3.50,0.015797


## **e) SQL-Like Operations:**

In [161]:
# Select
selected_df = df[['Name', 'Location', 'Price', 'Mileage', 'Power', 'Mileage_per_CC']]
selected_df.head()

,Name,Location,Price,Mileage,Power,Mileage_per_CC
0,Hyundai Creta 1.6 CRDi SX Option,Pune,12.50,19.67,126.20,0.012434
1,Honda Jazz V,Chennai,4.50,13.00,88.70,0.010842
2,Maruti Ertiga VDI,Chennai,6.00,20.77,88.76,0.016643
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,17.74,15.20,140.80,0.007724
4,Nissan Micra Diesel XV,Jaipur,3.50,23.08,63.10,0.015797


In [162]:
# Filter rows where Mileage is greater than 15
filtered_df = selected_df[selected_df['Mileage'] > 15]
filtered_df.head()


,Name,Location,Price,Mileage,Power,Mileage_per_CC
0,Hyundai Creta 1.6 CRDi SX Option,Pune,12.50,19.67,126.20,0.012434
2,Maruti Ertiga VDI,Chennai,6.00,20.77,88.76,0.016643
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,17.74,15.20,140.80,0.007724
4,Nissan Micra Diesel XV,Jaipur,3.50,23.08,63.10,0.015797
6,Volkswagen Vento Diesel Comfortline,Pune,5.20,20.54,103.60,0.012854


In [163]:
# Rename column
renamed_df = filtered_df.rename(columns={'Power': 'Horsepower'})
renamed_df.head()

,Name,Location,Price,Mileage,Horsepower,Mileage_per_CC
0,Hyundai Creta 1.6 CRDi SX Option,Pune,12.50,19.67,126.20,0.012434
2,Maruti Ertiga VDI,Chennai,6.00,20.77,88.76,0.016643
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,17.74,15.20,140.80,0.007724
4,Nissan Micra Diesel XV,Jaipur,3.50,23.08,63.10,0.015797
6,Volkswagen Vento Diesel Comfortline,Pune,5.20,20.54,103.60,0.012854


In [164]:
# Arrange data by Mileage in descending order
arranged_df = renamed_df.sort_values(by='Mileage', ascending=False)
arranged_df.head()

,Name,Location,Price,Mileage,Horsepower,Mileage_per_CC
3472,Maruti Dzire AMT ZDI Plus,Chennai,8.90,28.4,73.75,0.022756
1877,Maruti Dzire ZDI Plus,Jaipur,8.20,28.4,73.75,0.022756
1471,Maruti Dzire AMT ZDI Plus,Hyderabad,9.15,28.4,73.75,0.022756
2437,Maruti Swift VDI,Pune,6.00,28.4,74.00,0.022756
2361,Maruti Dzire VDI,Hyderabad,4.00,28.4,73.75,0.022756


In [165]:
# Summarize by Location and calculate the average Power
summary_df = arranged_df.groupby('Location').agg({'Horsepower': 'mean'}).reset_index()
summary_df.head()

,Location,Horsepower
0,Ahmedabad,101.203506
1,Bangalore,110.233503
2,Chennai,97.864465
3,Coimbatore,106.029738
4,Delhi,101.568200
